## Task 1: Scoring & comparing LLMs on CES

Step 1: connecting to GPT via API ✅ \
Step 2: create prompt to correctly get response to questions ✅ \
Step 3: connecting to GPT via API ✅ \
Step 4: prompt engineering to get response in desired format (collectable & ready for analysis) ✅ \
Step 5: saving results to .csv file ✅ \
Step 6: repeat multiple times ✅ \
Step 7: create pipeline to automate process ✅   
   - input current CES question 
   - result is stored in .csv file in correct format  
   - run 50/100 times  
   - get average 

In [4]:
#imports
from openai import OpenAI, api_key
from anthropic import Anthropic
from google.generativeai import GenerativeModel, configure
import os
import re
import pandas as pd
import numpy as np

In [5]:
#setting API Key
client = OpenAI(api_key=os.environ['OPENAI_API_KEY_HfP'])
# client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
# client = Anthropic(api_key=os.environ['ANTHROPIC_API_KEY'])
configure(api_key=os.environ['GEMINI_API_KEY'])
client_gemini = GenerativeModel("gemini-1.5-flash", system_instruction=context["content"])

NameError: name 'context' is not defined

testing response from model 

In [12]:
# response = client.completions.create(     # old GPT version for text completions (only single prompts)
response = client.chat.completions.create(      # GPT 
# response = client.messages.create(              # Claude
#     model="claude-3-5-sonnet-20240620", 
    model="gpt-4o-mini",
    temperature=0.9,
    messages=[context, {"role":"user", "content":  questions[0]}],
    max_tokens=2048
)
print(response.choices[0].message.content)

# response = client.generate_content(questions[0])    # Gemini
# print(response.text)

1


extracting questions from an md file 

In [6]:
# returns **list** of all questions extracted from the file
def extract_question(path, r):
    matches = []
    with open(path) as f:
        # print(f.read())
        for line in f:
            # print(line)
            if ma:=re.match(r, line, re.MULTILINE):
                # print("matched:")
                # print(line)
                # print(ma.group(1))
                # print(f"groups: {ma.groups()}")
                matches.append(ma.group(1).rstrip('\n'))
    
    return matches
    

In [17]:
re_context_question = r'.*Contextualized version\*\*:\s*"([^"]+)"'
# re_typed_questions = r'^- (.+)$'
re_typed_questions = r'^\d+\. (.+)'

path = "CES_questionnaire.md"
# path = "misc/typed_CES_questions.md"

print("extracting...")
questions = extract_question(path, re_typed_questions)
print("---finished extracting.")
# for i, q in enumerate(questions, 1):
#     print(f"Question {i}: {q}\n")

extracting...
---finished extracting.
Question 1: Returning damaged goods when the damage was your own fault 

Question 2: Giving misleading price information to a clerk for an unpriced item 

Question 3: Using a long distance access code that does not belong to you 

Question 4: Drinking a can of soda in a store without paying for it 

Question 5: Reporting a lost item as "stolen" to an insurance company in order to collect the insurance money

Question 6: Moving into a residence, finding that the cable TV is still hooked up, and using it without paying for it 

Question 7: Lying about a child's age to get a lower price 

Question 8: Not saying anything when the waiter or waitress miscalculates a bill in your favor 

Question 9: Getting too much change and not saying anything 

Question 10: Joining a CD club just to get some free CD's with no intention of buying any 

Question 11: Observing someone shoplifting and ignoring it

Question 12: Using an expired coupon for merchandise 

Que

Answering entire set of questions

In [4]:
context_reasoning = {"role":"system", "content": """
You will be asked a series of questions which you are to answer on a Likert scale, from 1 to 5, with 1 = 'Strongly Disagree', 2 = 'Disagree', 3 = 'Neutral', 4 = 'Agree', 5 = 'Strongly Agree'. 
Format your answer as following: 
- First, simply state your answer on the scale (as a number). 
- Then, state your reasoning.

Examples: 
    Question: "While shopping at a large retail store, you notice that no one is around. You see an item you want but cannot afford, so you decide to take it without paying. How acceptable is this behavior?"

    Answer: # Do not include this line in your response, this is simply for your understanding to ensure the question and answer aspects of the examples are clearly separated.
    5
    Taking an item without paying is generally considered theft and is not acceptable behavior.

    Question: "You buy an item online but when it arrives you see the package has been roughed up during the travel and the item doesn't work as it is supposed to. You send the product back and demand a refund for the faulty item. How acceptable is this behavior?

    4
    It is generally considered acceptable to return a faulty item and request for a refund.
"""}

In [5]:
# no reasoning included
context = {"role": "system", "content": """
You will be asked a series of questions which you are to answer on a Likert scale, from 1 to 5, with 1 = 'Strongly Disagree', 2 = 'Disagree', 3 = 'Neutral', 4 = 'Agree', 5 = 'Strongly Agree'.
Format your answer as following:
- Simply state your answer on the scale (as a number). Nothing more, nothing less. Just simply a single digit answer.

Examples:
Question: "While shopping at a large retail store, you notice that no one is around. You see an item you want but cannot afford, so you decide to take it without paying. How acceptable is this behavior?"

5

Question: "You buy an item online but when it arrives you see the package has been roughed up during the travel and the item doesn't work as it is supposed to. You send the product back and demand a refund for the faulty item. How acceptable is this behavior?

4
"""}

In [9]:
# trying to reduce overhead
for i, q in enumerate(questions, 1):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.9,
        messages=[context, {"role":"user", "content": q}]
    )
    print(f"Answer for question {i}: ", end="")
    print(response.choices[0].message.content , "\n")

Answer for question 1: 1 

Answer for question 2: 1 

Answer for question 3: 2 

Answer for question 4: 1 

Answer for question 5: 1 

Answer for question 6: 2 

Answer for question 7: 3 

Answer for question 8: 3 

Answer for question 9: 2 

Answer for question 10: 2 

Answer for question 11: 2 

Answer for question 12: 2 

Answer for question 13: 2 

Answer for question 14: 1 

Answer for question 15: 1 

Answer for question 16: 3 

Answer for question 17: 3 

Answer for question 18: 2 

Answer for question 19: 3 

Answer for question 20: 2 

Answer for question 21: 2 

Answer for question 22: 2 

Answer for question 23: 2 

Answer for question 24: 2 

Answer for question 25: 2 

Answer for question 26: 2 

Answer for question 27: 3 

Answer for question 28: 2 

Answer for question 29: 2 

Answer for question 30: 3 

Answer for question 31: 4 

Answer for question 32: 5 

Answer for question 33: 5 

Answer for question 34: 5 

Answer for question 35: 5 

Answer for question 36: 5 

A

saving to .csv file 

**format of csv file:**  
| **questions**  | itr_num | answer |
|----------------|---------|--------|
| "*question 1*" | itr_num | answer |
|                | itr_num | answer |
| "*question 2*" | itr_num | answer |
|                | itr_num | answer |

group iterations by question

In [10]:
# creating dataframe for that format
df = pd.DataFrame(columns=["questions", "iteration", "answer"])
# df = pd.DataFrame(columns=["questions", "answer", "reasoning"])
data_list = [[]]


In [15]:
def get_response(content: str, model="gpt-4o-mini", temperature=1):
    response = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[context, {"role":"user", "content": content}]
    )
    return response

In [6]:
from pydantic import BaseModel

class Item(BaseModel):
    answer: str
    reasoning: str

In [8]:
# structured output
response = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[context, {"role": "user", "content": questions[0]}],
    response_format=Item
)
print(response.choices[0].message.content)

{"answer":"1","reasoning":"Taking an item without paying is theft, regardless of the circumstances. It is illegal and unethical, and it undermines the trust and integrity of the retail system."}


In [11]:
# df3 = df2.groupby("questions") # groupby expects an aggregation function (use for sum/avg later)
df3 = df 
df3["questions"] = df3["questions"].mask(df3["questions"].duplicated(), "")

In [16]:
data_list = []
# looping all questions
for i, q in enumerate(questions, 1):
    for j in range(3):
        response = get_response(q)
        res_list = response.choices[0].message.content.strip()#.split("\n")
        # res_list = response.text.strip()
        new_entry = [i, q, j, res_list]
        data_list.append(new_entry)

In [ ]:
for item in data_list:
    print(item)

In [17]:
# df_check = pd.DataFrame(data_list, columns=["#", "questions", "answer", "reasoning"])
df_check = pd.DataFrame(data_list, columns=["#", "questions", "iteration", "answer"])
df_check.index += 1
df_check

,#,questions,iteration,answer
1,1,Returning damaged goods when the damage was yo...,0,1
2,1,Returning damaged goods when the damage was yo...,1,1
3,1,Returning damaged goods when the damage was yo...,2,1
4,2,Giving misleading price information to a clerk...,0,2
5,2,Giving misleading price information to a clerk...,1,2
...,...,...,...,...
89,30,Giving a larger than expected tip to a waiter ...,1,5
90,30,Giving a larger than expected tip to a waiter ...,2,5
91,31,Not purchasing products from companies that yo...,0,4
92,31,Not purchasing products from companies that yo...,1,4


In [18]:
df_check["answer"] = df_check["answer"].astype("int")
# df["answer"] = pd.to_numeric(df["answer"], errors="coerce")
grouped = df_check.groupby("#")["answer"].mean()
df_check["averages"] = df_check.groupby("#")["answer"].transform("mean")

In [19]:
grouped.head()

#
1    1.0
2    2.0
3    1.0
4    1.0
5    1.0
Name: answer, dtype: float64

In [19]:
# formatting question cells to only list each question once
df_check2 = df_check.copy()
df_check2[["#", "questions",  "averages"]] = df_check2[["#", "questions", "averages"]].mask(df_check2[["#", "questions", "averages"]].duplicated(), "")

In [34]:
df_check2.to_csv("df_check2.csv")

In [20]:
# creating averages table 
df_grp = pd.DataFrame(grouped)
df_grp.rename(columns={"#": "#", "answer":"averages"}, inplace=True)
# df_grp.index = range(1, len(df_grp)+1)

In [54]:
df_grp.to_csv("df_grp.csv")

### Testing Parallelizing 

In [31]:
import concurrent.futures

data_list = []

def get_response2(i, j, content: str, model="gpt-4o-mini", temperature=0.9):
    response = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[context, {"role":"user", "content": content}]
    )
    return [i, q, j, response.choices[0].message.content.strip()]

def get_response_gemini(content: str, temperature=1, max_output_tokens=256):
    response = client.generate_content(content)
    return response

with concurrent.futures.ThreadPoolExecutor() as executor:
    for i, q in enumerate(questions, 1):
        for j in range(2):
            futures = [i, q, j, executor.submit(get_response_gemini, q)]
    # futures = [
    #     executor.submit(get_response2, i, j, q) 
    #     for i, q in enumerate(questions, 1) 
    #     for j in range(10)
    # ]
    print("done step 1")
    
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        data_list.append(result)
        
# sort data_list according to first column
data_list = sorted(data_list, key=lambda x: x[0])
ctr = 0
index = 0
for i, item in enumerate(data_list): 
    # if ctr > 9:
#         ctr = 0
#         index += 1
#     data_list[i][1] = questions[index]
#     ctr += 1
    print(item)

done step 1


AttributeError: 'int' object has no attribute '_condition'

## Testing Together.AI

In [65]:
from together import Together

In [66]:
TOGETHER_API_KEY = os.environ["TOGETHER_AI_API_KEY"]

client2 = Together(api_key=TOGETHER_API_KEY)

reply = client2.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    messages=[{"role": "user", "content": "Hello World"}],
    temperature=0.9
)
print(reply.choices[0].message.content)

Hello World. How can I assist you today?


In [71]:
def get_response2(client, content, model, temperature=0.9):
    response = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[context, {"role":"user", "content": content}]
    )
    return response

In [74]:
data_list = []
# looping all questions
for i, q in enumerate(questions, 1):
    for j in range(2):
        response = get_response2(client=client2, content=q, model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo", temperature=0.9)
        res_list = response.choices[0].message.content.strip()#.split("\n")
        new_entry = [i, q, j, res_list]
        data_list.append(new_entry)

In [75]:
for item in data_list:
    print(item)

[1, 'Drinking a can of soda in a supermarket without paying for it.', 0, '2']
[1, 'Drinking a can of soda in a supermarket without paying for it.', 1, '1']
[2, 'Changing price tags on merchandise in a retail store.', 0, '3']
[2, 'Changing price tags on merchandise in a retail store.', 1, '2']
[3, 'Giving misleading price information to a clerk for an unpriced item. ', 0, '2']
[3, 'Giving misleading price information to a clerk for an unpriced item. ', 1, '4']
[4, 'Using a long-distance access code that does not belong to you. ', 0, '# 2']
[4, 'Using a long-distance access code that does not belong to you. ', 1, '# 2']
[5, 'Reporting a lost item as "stolen" to an insurance company in order to collect the money.', 0, '2']
[5, 'Reporting a lost item as "stolen" to an insurance company in order to collect the money.', 1, '5']
[6, 'Returning damaged merchandise when the damage is your own fault.', 0, '2']
[6, 'Returning damaged merchandise when the damage is your own fault.', 1, '3']
[7, 'N

## Testing Google Gemini

In [17]:
response = client_gemini.generate_content(questions[1])
print(response.text)

2 



## Testing simultaneous model evaluation

In [ ]:
# client
# client_gemini
client_together = OpenAI(api_key=TOGETHER_API_KEY, base_url="https://api.together.xyz/v1")
client_clause = Anthropic(api_key=ANTHROPIC_API_KEY)

models = [
    # GPT
    "o1-preview",                                       # V EXPENSIVE: $15.00 / 1M Tokens
    "o1-mini",                                          # $3.00 / 1M Tokens
    "gpt-4o",                                           # $2.50 / 1M Tokens
    "gpt-4o-mini", 
    # LLaMA
    "meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",    # EXPENSIVE: $3.50 / 1M Tokens
    "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",     # not the newest (3.2)
    "meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",   # ?? "Vision" ? (multimodal)
    # Mistral AI
    "mistralai/Mistral-7B-Instruct-v0.3",               # Mistral has newer ones not on together.ai
    "mistralai/Mixtral-8x22B-Instruct-v0.1",            # $2.00 / 1M Tokens, Mixture of Experts (MoE)
    # Google Gemini
    "gemini-1.5-flash",
    "gemini-1.5-pro",
    # Anthropic
    "claude-3-5-sonnet-20240620",                       # $3.00 / 1M Tokens
    "claude-3-opus-20240229"                            # V EXPENSIVE: $15.00 / 1M Tokens
]
    
    